In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_parquet('../data/processed/m5_timeseries.parquet')
df = df.sort_values(['id', 'date'])

In [ ]:
#Creating Lags
lags = [1, 7, 14, 28]

for lag in lags:
    df[f'[lags_{lag}'] = df.groupby('id')['sales'].shift(lag)

In [15]:
df.shape

(1530400, 26)

In [13]:
(df['sales'] != 0).sum()

np.int64(476556)

In [16]:
#Creating Rolling / Moving Window

windows = [7, 14, 28]
for w in windows:
    grp = df.groupby('id')['sales']
    df[f'rolling_mean_{w}'] = grp.shift(1).rolling(w).mean()
    df[f'rolling_std_{w}'] = grp.shift(1).rolling(w).std()

In [17]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'dayofweek', 'weekofyear', 'month', 'year', '[lags_1',
       '[lags_7', '[lags_14', '[lags_28', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_14', 'rolling_std_14', 'rolling_mean_28',
       'rolling_std_28'],
      dtype='object')

In [18]:
df['quarter'] = df['date'].dt.quarter
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

In [19]:
df['dow_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

In [21]:
df['price_max'] = df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')
df['price_min'] = df.groupby(['store_id', 'item_id'])['sell_price'].transform('min')
df['price_norm'] = (df['sell_price'] - df['price_min']) / (df['price_max'] - df['price_min'])
df['price_change'] = df.groupby(['store_id','item_id'])['sell_price'].pct_change()

C:\Users\srmpc\AppData\Local\Temp\ipykernel_16328\180819684.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['price_change'] = df.groupby(['store_id','item_id'])['sell_price'].pct_change()


In [22]:
df['is_event'] = (df['event_name_1'] != None).astype(int)
df['is_snap_CA'] = df['snap_CA']
df['is_snap_TX'] = df['snap_TX']
df['is_snap_WI'] = df['snap_WI']

In [24]:
#Lag or price and Rolling price mean
df['lag_price_7'] = df.groupby(['store_id','item_id'])['sell_price'].shift(7)
df['rmean_price_28'] = df.groupby(['store_id','item_id'])['sell_price'].shift(1).rolling(28).mean()

In [26]:
for col in ['item_id','dept_id','cat_id','store_id','state_id']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [27]:
df['sales_log1p'] = np.log1p(df['sales'])

In [28]:
stats = df.groupby('id')['sales'].agg(['mean','std']).reset_index()
stats['cv'] = stats['std']/stats['mean']
stats.rename(columns={'cv':'demand_volatility'}, inplace=True)
df = df.merge(stats[['id', 'demand_volatility']], on='id', how='left')

In [31]:
df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'dayofweek', 'weekofyear', 'month', 'year', '[lags_1',
       '[lags_7', '[lags_14', '[lags_28', 'rolling_mean_7', 'rolling_std_7',
       'rolling_mean_14', 'rolling_std_14', 'rolling_mean_28',
       'rolling_std_28', 'quarter', 'is_weekend', 'dow_sin', 'dow_cos',
       'price_max', 'price_min', 'price_norm', 'price_change', 'is_event',
       'is_snap_CA', 'is_snap_TX', 'is_snap_WI', 'lag_price_7',
       'rmean_price_28', 'sales_log1p', 'demand_volatility'],
      dtype='object')

In [33]:
df['rolling_mean_7_lag_7_ratio'] = df['rolling_mean_7'] / (df['[lags_7'] + 1)

In [35]:
#Handling Missing Values & Data Leakage
df = df.dropna(subset=['[lags_28'])
df.fillna(0, inplace=True)

C:\Users\srmpc\AppData\Local\Temp\ipykernel_16328\1808371811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


In [36]:
#feature validation and sanity check
print(df.shape)
df.head(10)
print(df.isnull().mean().sort_values(ascending=False).head(10))

(1508000, 49)
id          0.0
item_id     0.0
dept_id     0.0
cat_id      0.0
store_id    0.0
state_id    0.0
d           0.0
sales       0.0
date        0.0
wm_yr_wk    0.0
dtype: float64


In [38]:
df.to_parquet('../data/processed/m5_features.parquet')

In [39]:
df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,price_change,is_event,is_snap_CA,is_snap_TX,is_snap_WI,lag_price_7,rmean_price_28,sales_log1p,demand_volatility,rolling_mean_7_lag_7_ratio
1530395,HOUSEHOLD_1_239_CA_1_validation,799,2,1,0,0,d_1909,0,2016-04-20,11612,...,0.0,1,0,0,0,3.97,3.97,0.000000,2.489898,0.500000
1530396,HOUSEHOLD_1_239_CA_1_validation,799,2,1,0,0,d_1910,0,2016-04-21,11612,...,0.0,1,0,0,0,3.97,3.97,0.000000,2.489898,0.428571
1530397,HOUSEHOLD_1_239_CA_1_validation,799,2,1,0,0,d_1911,3,2016-04-22,11612,...,0.0,1,0,0,0,3.97,3.97,1.386294,2.489898,0.357143
1530398,HOUSEHOLD_1_239_CA_1_validation,799,2,1,0,0,d_1912,2,2016-04-23,11613,...,0.0,1,0,0,0,3.97,3.97,1.098612,2.489898,0.333333
1530399,HOUSEHOLD_1_239_CA_1_validation,799,2,1,0,0,d_1913,0,2016-04-24,11613,...,0.0,1,0,0,0,3.97,3.97,0.000000,2.489898,0.333333


In [40]:
cutoff_date = '2016-03-23'
train = df[df['date'] <= cutoff_date]
valid = df[df['date'] > cutoff_date]

train.to_parquet('../data/processed/train.parquet')
valid.to_parquet('../data/processed/valid.parquet')